# Threat Hunting In Rapid Iterations

    Author: Alyssa Rahman (@ramen0x3f)
    Last Updated: 2022-02-02
    
The THIRI notebook is designed as a research aide to let you rapidly prototype threat hunting rules.

## Getting Started
Welcome to the **Threat Hunting In Rapid Iterations (THIRI)** Jupyter notebook! 

The THIRI notebook is designed as a research aide to let you rapidly prototype threat hunting rules.

To get started, click **run**.

*Jupyter Pro Tip:*
Enable NBExtensions to use the **"Hide Code Input"** (eye icon) and **"Run initialization cells"** (calculator icon) options above. (Optional, but it will make your output cleaner.)

### Select Engine
Select an engine from the dropdown below. "Engines" can be thought of as indicator types or templates. 

THIRI dynamically loads all sub-classes of Engine() in the utils/engines.py module. 

In [1]:
from inspect import getmembers, isclass
from IPython.display import clear_output, Javascript, Markdown
from ipywidgets import widgets, VBox, HBox, Label
from json import loads
from sys import stderr
from typing import Union
import utils.engines

#GLOBAL variables used as gates to keep later cells from executing prematurely
GLOBAL = dict.fromkeys(["selected_engine", "engine", #Engine
                        "inputs", "clean_options", #Engine input/options
                        "edit_boxes", "clean_edits"], None) #Rule edits

#Helper function to reset global variables
def clear_vars(exceptions = []):
    global GLOBAL
    #Clearing variables
    for x,y in GLOBAL.items():
        x = y if x in exceptions else None

#Helper function to move to the next cell    
def next_cell(e):
    display(Javascript('IPython.notebook.select_next()'))
    display(Javascript('IPython.notebook.execute_selected_cells()'))
    
#Helper function called when a new engine is selected
def fresh_engine(e):
    #Reset vars
    clear_vars(["selected_engine"])
    
    #Clear rules from last cell
    display(Javascript('IPython.notebook.select(9)'))
    display(Javascript('IPython.notebook.execute_selected_cells()'))

    #Clear inputs
    display(Javascript('IPython.notebook.select(6)'))
    display(Javascript('IPython.notebook.execute_selected_cells()'))

    #Move to next cell
    display(Javascript('IPython.notebook.select(5)'))
    display(Javascript('IPython.notebook.execute_selected_cells()'))
    
#Dynamically build dropdown of classes from utils.engines
GLOBAL["selected_engine"] = widgets.Dropdown(
    options=[e for e in getmembers(utils.engines, isclass) 
             if 'utils.engines' in str(e[1]) and e[0] not in ['ABC', 'Engine', 'TemplateClass']],
    description='Engine',
    disabled=False,
    value=None
)

#Add callback helper and display
GLOBAL["selected_engine"].observe(fresh_engine)
display(GLOBAL["selected_engine"])

Dropdown(description='Engine', options=(('CodeCert', <class 'utils.engines.CodeCert'>), ('DLLHijack', <class '…

### Provide Inputs
Each engine has **custom options** and **formats** they support. 

Some engines (e.g. SerializedObject) may first let you **toggle** select a sub-type of rule to generate. (e.g. inputting a Keyword vs a Chain of Keywords)

Fill out the form, and a **prototype rule** will be generated!

In [3]:
#Guard condition in case the helper function progresses too fast
if GLOBAL["selected_engine"] is not None and GLOBAL["selected_engine"].value is not None:
    #Instantiate engine and get first round of inputs
    GLOBAL["engine"] = GLOBAL["selected_engine"].value()
    opts, tips, descr = GLOBAL["engine"].get_input_type()

    #Display engine description
    display(Markdown(descr))
    
    #Move to next cell if no options provided
    if opts is None:
        search_type = None
        next_cell(None)

    #Display toggle options if provided
    else:    
        search_type = widgets.ToggleButtons(
            options=opts,
            description='Type:',
            disabled=False,
            tooltips=tips,
            value=None
        )
        
        #Add callback helper and display
        search_type.observe(next_cell)
        display(search_type)

In [4]:
#Guard condition in case the helper function progresses too fast
if GLOBAL["engine"] is not None:
    #Get custom input/options from engine
    #Overwrites search_type.value in case it wasn't provided
    st = search_type.value if search_type is not None else ""
    custom_options = GLOBAL["engine"].get_custom_options(st)

    #Display custom options
    GLOBAL["inputs"] = []
    for opt in custom_options:
        desc = Label(opt.description, layout={'width':'10%'})
        opt.description = ""
        
        #Can't set Text, Textarea, or SelectMultiple widgets to None
        if not isinstance(opt, Union[widgets.Text, widgets.Textarea, widgets.SelectMultiple]):
            opt.value = None

        #Add calback helper and create box with label
        opt.observe(next_cell)
        GLOBAL["inputs"].append(HBox([desc, opt]))
        
    #Put that box inside another box and then mail that box to myself
    display(VBox(GLOBAL["inputs"]))

### Edit Rule
All generated rules will be **displayed** below. 

Make any changes you'd like in the **Edit** box. 

In [5]:
#Guard condition in case the helper function progresses too fast
GLOBAL["clean_options"] = {i.children[0].value:i.children[1].value 
                               for i in GLOBAL["inputs"]} if GLOBAL["inputs"] is not None else {}

if GLOBAL["engine"] and len(GLOBAL["clean_options"]['Format']) > 0:
    #Generate rule
    try: 
        rules = GLOBAL["engine"].generate(GLOBAL["clean_options"])
        
        #Iterate
        GLOBAL["edit_boxes"] = []
        for rulename, rulecontent in rules:
            #Create input to edit rule
            desc = rulename if rulename is not None else 'unnamed rule'
            rule = widgets.Textarea(
                disabled=False,
                layout={'width': '60%'},
                value=rulecontent
            )

            rule.observe(next_cell)
            GLOBAL["edit_boxes"].append(HBox([Label(desc, layout={'width':'30%'}), rule]))

        #Put that box inside another box and then mail that box to myself
        display(VBox(GLOBAL["edit_boxes"]))
    
    except RuntimeError as r:
        print(f"[!] {r}")

### Final Prototype
Your final rule prototype will be **printed** below (with any edits automatically rendered). 

Future plans include automated **publishing** to resources such as VirusTotal, Github, etc. 

In [6]:
#Guard condition in case the helper function progresses too fast
GLOBAL["clean_edits"] = {e.children[0].value:e.children[1].value 
                   for e in GLOBAL["edit_boxes"]} if GLOBAL["edit_boxes"] is not None else {}

if not any(y is None or len(y) == 0 for x,y in GLOBAL["clean_edits"].items()):
    #Just print for now
    for rulename, rulecontent in GLOBAL["clean_edits"].items():
        print(f"\n---------------------------\n{rulename}\n---------------------------\n{rulecontent}\n")